## Hypothesis Testing

H0: Similarity between benchmark/original set of passthought and successive pass-thoughts increases with positive feedback  compared to negative feedback
across all fft channels

In [2]:
%matplotlib inline 
#Importations
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial import distance
from scipy.stats import ttest_ind, chisquare


In [3]:
#Variables
folder = "../../data_mrt/"
files = ['always_accept_1.csv','always_accept_2.csv','always_accept_3.csv','always_accept_4.csv',
        'always_reject_1.csv','always_reject_2.csv','always_reject_3.csv','always_reject_4.csv',]
data = []
times = []
experiment_stages = ['calibration','set_passthought','authenticate'] # This experiment had 3 stages,  
steps = [1,3,4] #Calibration has 1 step, set_passthought has 3 and auth has 4
passthougt_duration = 8 #passthought last 8 seconds
conditions = []
substeps = []
ControlnTreatment=[] #Stores which subjects are in always accept and which ones are in always reject
markers = [' /Marker/BLINK',
           ' /Marker/CalibrationComplete', 
           ' /Marker/ThinkPassthought',
           ' /Marker/BLANK',
           ' /Marker/YouhavesuccessfullysetyourPassthought',
           ' /Marker/ThinkPassthoughttoAunthenticate', 
           ' /Marker/Classifying',
           ' /Marker/AuthenticationSuccessful',
           ' /Marker/AuthenticationFailed'
       ]
ffts = [' /muse/elements/raw_fft0',' /muse/elements/raw_fft1',
        ' /muse/elements/raw_fft2',' /muse/elements/raw_fft3']

stages, allsteps, start, end, raw_fft,fft_type,subjects,alldata =[],[],[],[],[],[],[],[]


In [4]:
#Read data into pandas
for file in files:
    print("Appending file : ",file)
    filedata = pd.read_csv(folder+file)
    data.append(filedata)

Appending file :  always_accept_1.csv
Appending file :  always_accept_2.csv


//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Appending file :  always_accept_3.csv
Appending file :  always_accept_4.csv


//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Appending file :  always_reject_1.csv
Appending file :  always_reject_2.csv
Appending file :  always_reject_3.csv
Appending file :  always_reject_4.csv


In [5]:
#data[:5]

In [6]:
#Methods

#Given two vectors this function returns the cosine similarity
def cosine(X,Y):
    dist = distance.cosine(X,Y)
    return dist


#Function that compares the cosine similarity between 2 groups
def cos_compare(fft_type, step,substep,subject):
    #Set-Passthought will be our comparison session
    #Select all fft segnals in that period and get the mean across all 129 
    set_passthought_signals = df[(df['stage']=='set_passthought') 
                                 &(df['fft_type'] == fft_type)#' /muse/elements/raw_fft1')
                                 &(df['subject'] == subject)]['raw_fft']

    #put mean signals from the 3 set passthought timeperiods into one dataframe
    mylist = [x.applymap(float).mean(axis=0) for x in set_passthought_signals ] 
    dataframe = pd.DataFrame(data=mylist) #Convert the list to a dataframe
    benchmark = dataframe.mean(axis=0) #Take the mean across the 3 sessions
    benchmark.shape

    
#     comparison_group = (df[(df['stage']=="authenticate") &(df['fft_type'] == fft_type) 
#                            &(df['step'] == step) &(df['subject'] == subject)] )['raw_fft']
    
    comparison_group = (df[(df['stage']=="authenticate") &(df['fft_type'] == fft_type) 
                           &(df['substep'] == substep) 
                           &(df['step'] == step) &(df['subject'] == subject)] )['raw_fft']    
    
    mylist2 = [x.applymap(float).mean(axis=0) for x in comparison_group ] 
    dataframe2 = pd.DataFrame(data=mylist2) #Convert the list to a dataframe
    comparison_group = dataframe2.mean(axis=0) #Take the mean across the 3 sessions
    comparison_group.shape
    return cosine(benchmark,comparison_group)

def get_markers(mydata):
    my_dict = {}
    always_accept=None
    for marker in markers:
        #extract the time when the marker was displayed
        time = mydata.loc[mydata["task"] == marker]['time']
        #Save marker and times 
        #print(marker,list(time))
        my_dict[marker] = list(time)
        
    if len(my_dict[' /Marker/BLANK'])>2:
        my_dict[' /Marker/BLANK'] = my_dict[' /Marker/BLANK'][1:-1]
        
    #Decide which condition the user belongs to
    if len(my_dict[' /Marker/AuthenticationSuccessful'])>0:
        always_accept=1
    if len(my_dict[' /Marker/AuthenticationFailed'])>0:
        always_accept=0      

    my_dict['set_passthought_marker'] = my_dict[' /Marker/BLANK'] + my_dict[' /Marker/YouhavesuccessfullysetyourPassthought']

    return my_dict,always_accept


def filtering(start_time,end_time,f,mydata):
    filtered = mydata[(mydata["time"] >= start_time) & (mydata["time"] <= end_time)] #filter by time  
    
    filtered_fft =filtered[filtered["task"]== f] #filter the relevant fft 
    fft= filtered_fft.iloc[:,2:131]#select the 129 fft columns from the 2nd one to the end
    
    return fft

def append_elements(stage,s,fft,start_time,end_time,f,subject,condition,substep):
    stages.append(stage)
    allsteps.append(s)
    raw_fft.append(fft)
    start.append(start_time)
    end.append(end_time)
    fft_type.append(f)
    subjects.append(subject)
    conditions.append(condition)
    substeps.append(substep)
    
def extract_fft(subjectID,results, mydata):
    my_dict = results[0]
    condition = results[1] #2nd tuple element is the condition-always accept or always reject
    #extract all the fft signals for each stage
    for stage in experiment_stages:
        n_steps = steps[experiment_stages.index(stage)] #no of steps per stage
        for s in range(n_steps):
            for f in ffts:
                if stage == "calibration":
                    start_marker = ' /Marker/BLINK'
                    end_marker = ' /Marker/CalibrationComplete'
                    substep(stage,s,f,subjectID,condition,start_marker,end_marker,mydata,my_dict)
                    
                elif stage == "set_passthought":
                    
                    start_marker = ' /Marker/ThinkPassthought'
                    end_marker = 'set_passthought_marker'
                    substep(stage,s,f,subjectID,condition,start_marker,end_marker,mydata,my_dict)
                    
                elif stage == "authenticate":
                    start_marker = ' /Marker/ThinkPassthoughttoAunthenticate'
                    end_marker = ' /Marker/Classifying'
                    substep(stage,s,f,subjectID,condition,start_marker,end_marker,mydata,my_dict)
                    
                else:
                    print('Error!!!')
                    
no_of_substeps = 4
increment = 2 #2seconds
#Create a function that splits each step into substeps
def substep(stage,s,f,subjectID,condition,start_marker,end_marker,mydata,my_dict):
    if stage == "authenticate":
        stage_start_time = my_dict[start_marker][s]
        stage_end_time   = my_dict[end_marker][s]
        for substep in range(no_of_substeps):       
            start_time = stage_start_time +substep*increment
            end_time = start_time+increment
            fft = filtering(start_time,end_time,f,mydata)
            append_elements(stage,s,fft,start_time,end_time,f,subjectID,condition,substep) 
    else:
        substep=None
        start_time = my_dict[start_marker][s]
        end_time   = my_dict[end_marker][s]
        fft = filtering(start_time,end_time,f,mydata)
        append_elements(stage,s,fft,start_time,end_time,f,subjectID,condition,substep)

In [7]:
for subjectID in range(len(data)):
    mydata=data[subjectID]
    results = get_markers(mydata)
    extract_fft(subjectID,results, mydata)
    ControlnTreatment.append([subjectID,results[1]]) #Store subjectID, and whether the person is in control/treatment

    
d = {'stage': stages, 'step': allsteps,'substep':substeps,
     'start':start ,'end':end, 
     'fft_type':fft_type,'raw_fft':raw_fft,'subject':subjects,'conditions':conditions}

df = pd.DataFrame(data=d, columns=['stage','step','substep','fft_type','start','end','raw_fft','subject','conditions'])
df.head()   


,stage,step,substep,fft_type,start,end,raw_fft,subject,conditions
0,calibration,0,NaN,/muse/elements/raw_fft0,1515093498,1515093508,v1 v2 v3 ...,0,1
1,calibration,0,NaN,/muse/elements/raw_fft1,1515093498,1515093508,v1 v2 v3 ...,0,1
2,calibration,0,NaN,/muse/elements/raw_fft2,1515093498,1515093508,v1 v2 v3 ...,0,1
3,calibration,0,NaN,/muse/elements/raw_fft3,1515093498,1515093508,v1 v2 v3 ...,0,1
4,set_passthought,0,NaN,/muse/elements/raw_fft0,1515093583,1515093591,v1 v2 v3 ...,0,1


In [8]:
#df['raw_fft'][0]

In [9]:
df.shape

(640, 9)

In [10]:
df[df['stage'] =='authenticate'].head()

,stage,step,substep,fft_type,start,end,raw_fft,subject,conditions
16,authenticate,0,0.0,/muse/elements/raw_fft0,1515093676,1515093678,v1 v2 v3 ...,0,1
17,authenticate,0,1.0,/muse/elements/raw_fft0,1515093678,1515093680,v1 v2 v3 ...,0,1
18,authenticate,0,2.0,/muse/elements/raw_fft0,1515093680,1515093682,v1 v2 v3 ...,0,1
19,authenticate,0,3.0,/muse/elements/raw_fft0,1515093682,1515093684,v1 v2 v3 ...,0,1
20,authenticate,0,0.0,/muse/elements/raw_fft1,1515093676,1515093678,v1 v2 v3 ...,0,1


In [11]:
cosinedata = []
for subjectID in range(len(data)):
    for f in ffts:
        d = [cos_compare(f,step,substep,subjectID) for step in range(steps[2]) for substep in range(4) ]
        d.append(subjectID)
        d.append(f)
        cosinedata.append(d) 

In [12]:
#cosinedata
steps

[1, 3, 4]

In [13]:
cosinedata[0]

[0.036718598048823825,
 0.035225827599415283,
 0.017911898924501624,
 0.014399442080735514,
 0.018881578093808415,
 0.025725326629077006,
 0.029749225313634553,
 0.015205986642908842,
 0.021740614156051774,
 0.0130266558444736,
 0.025006831296776055,
 0.021541254722739933,
 0.013967452416141168,
 0.016726121174738484,
 0.017766512270352863,
 0.013090531564528662,
 0,
 ' /muse/elements/raw_fft0']

In [14]:
ControlnTreatmentdf = pd.DataFrame(data=ControlnTreatment,columns=['Subject', 'Condition'])
ControlnTreatmentdf.head()

,Subject,Condition
0,0,1
1,1,1
2,2,1
3,3,1
4,4,0


In [15]:
df2 = pd.DataFrame(data=cosinedata, columns=['A1_1','A1_2','A1_3','A1_4','A2_1','A2_2','A2_3','A2_4',
                                             'A3_1','A3_2','A3_3','A3_4','A4_1','A4_2','A4_3','A4_4',
                                             'Subject','fft_type'])
df2.head()

,A1_1,A1_2,A1_3,A1_4,A2_1,A2_2,A2_3,A2_4,A3_1,A3_2,A3_3,A3_4,A4_1,A4_2,A4_3,A4_4,Subject,fft_type
0,0.036719,0.035226,0.017912,0.014399,0.018882,0.025725,0.029749,0.015206,0.021741,0.013027,0.025007,0.021541,0.013967,0.016726,0.017767,0.013091,0,/muse/elements/raw_fft0
1,0.025387,0.023382,0.013207,0.014187,0.018166,0.014129,0.017312,0.020434,0.020742,0.009984,0.012688,0.017011,0.013046,0.015239,0.020661,0.014465,0,/muse/elements/raw_fft1
2,0.040136,0.036893,0.012782,0.013041,0.010399,0.014973,0.017125,0.018330,0.027292,0.009771,0.012608,0.016008,0.018071,0.014668,0.016626,0.017250,0,/muse/elements/raw_fft2
3,0.030161,0.031585,0.031143,0.018099,0.020021,0.027691,0.041291,0.020921,0.018974,0.019445,0.035708,0.024964,0.013161,0.021126,0.031414,0.021523,0,/muse/elements/raw_fft3
4,0.041704,0.022823,0.021784,0.022134,0.038744,0.016138,0.020015,0.015405,0.049833,0.017565,0.031265,0.038727,0.029822,0.022885,0.027171,0.025696,1,/muse/elements/raw_fft0


One Sample Vs Two Sample T-Test

A two-sample t-test investigates whether the means of two independent data samples differ from one another. In a two-sample test, the null hypothesis is that the means of both groups are the same. Unlike the one sample-test where we test against a known population parameter, the two sample test only involves sample means. 

In this case, we have no way of knowing what the population parameters are. We are interested in comparing 2 groups, always accept and always reject. We therefore conduct a 2 samples t-test.

Implementation of one and two sample t-test in python is available here:http://hamelg.blogspot.com/2015/11/python-for-data-analysis-part-24.html

To perform two-sample one-tailed t-test: 
https://stackoverflow.com/questions/15984221/how-to-perform-two-sample-one-tailed-t-test-with-numpy-scipy

http://evafengeva.blogspot.com/2017/09/how-to-interpret-results-of-two-sample.html

"because the one-sided tests can be backed out from the two-sided tests. (With symmetric distributions one-sided p-value is just half of the two-sided pvalue)
It goes on to say that scipy always gives the test statistic as signed. This means that given p and t values from a two-tailed test, you would reject the null hypothesis of a greater-than test when p/2 < alpha and t > 0, and of a less-than test when p/2 < alpha and t < 0"

In [17]:
df_merged = pd.merge(df2 , ControlnTreatmentdf, on='Subject',how="outer")
df_merged.head()

,A1_1,A1_2,A1_3,A1_4,A2_1,A2_2,A2_3,A2_4,A3_1,A3_2,A3_3,A3_4,A4_1,A4_2,A4_3,A4_4,Subject,fft_type,Condition
0,0.036719,0.035226,0.017912,0.014399,0.018882,0.025725,0.029749,0.015206,0.021741,0.013027,0.025007,0.021541,0.013967,0.016726,0.017767,0.013091,0,/muse/elements/raw_fft0,1
1,0.025387,0.023382,0.013207,0.014187,0.018166,0.014129,0.017312,0.020434,0.020742,0.009984,0.012688,0.017011,0.013046,0.015239,0.020661,0.014465,0,/muse/elements/raw_fft1,1
2,0.040136,0.036893,0.012782,0.013041,0.010399,0.014973,0.017125,0.018330,0.027292,0.009771,0.012608,0.016008,0.018071,0.014668,0.016626,0.017250,0,/muse/elements/raw_fft2,1
3,0.030161,0.031585,0.031143,0.018099,0.020021,0.027691,0.041291,0.020921,0.018974,0.019445,0.035708,0.024964,0.013161,0.021126,0.031414,0.021523,0,/muse/elements/raw_fft3,1
4,0.041704,0.022823,0.021784,0.022134,0.038744,0.016138,0.020015,0.015405,0.049833,0.017565,0.031265,0.038727,0.029822,0.022885,0.027171,0.025696,1,/muse/elements/raw_fft0,1


In [22]:
#Run a t-test
#-3 is because we ignore the last 3 dataframe columns showing subject,condition and fft
always_accept_group = df_merged[df_merged['Condition'] == 1].iloc[:,:-3].applymap(float).values.flatten()
always_reject_group = df_merged[df_merged['Condition'] == 0].iloc[:,:-3].applymap(float).values.flatten()
ttest_result = ttest_ind(always_accept_group,always_reject_group,equal_var = False)

In [23]:
real_p = ttest_result[1]/2
real_t_score = ttest_result[0]

In [24]:
real_p,real_t_score

(1.4579023729620564e-14, -8.0606943462103331)